# DungeonBreak Text Adventure (Game Traits)

Interactive notebook demo that consumes:
- `game_traits_manifest.json`
- `narrative_snapshot.json`

If snapshot forces/locations are missing, actions still run but warn and avoid fabricating trait shifts.


In [ ]:
import sys
from pathlib import Path

for base in [Path.cwd(), Path.cwd().parent]:
    if (base / 'src' / 'dungeonbreak_narrative').exists():
        sys.path.insert(0, str(base))
        break

from dungeonbreak_narrative import (
    EVENT_DIALOG,
    EVENT_REST,
    EVENT_TRAINING,
    EVENT_TRAVEL,
    SCENE_HOME_MERCHANT,
    SCENE_HOME_MOTHER,
    SCENE_TO_NPC,
    SCENE_TOWN_SQUARE,
    apply_one_event,
    available_dialogs,
    in_quest_region,
    initial_state,
    load_game_traits_manifest,
    load_narrative_snapshot,
    validate_game_alignment_warn_only,
)

import numpy as np
from ipywidgets import Button, HBox, HTML, Layout, Output, VBox


In [ ]:
alignment = validate_game_alignment_warn_only()
traits = load_game_traits_manifest()
snapshot = load_narrative_snapshot()

state = initial_state(scene_id=SCENE_TOWN_SQUARE, entity_name='Kaiza', snapshot=snapshot)
event_history = []

SCENE_DESCRIPTIONS = {
    SCENE_TOWN_SQUARE: 'You are in the town square.',
    SCENE_HOME_MOTHER: 'You are at home. Mother is here.',
    SCENE_HOME_MERCHANT: "You are at the merchant's.",
}
REACHABLE = {
    SCENE_TOWN_SQUARE: [SCENE_HOME_MOTHER, SCENE_HOME_MERCHANT],
    SCENE_HOME_MOTHER: [SCENE_TOWN_SQUARE],
    SCENE_HOME_MERCHANT: [SCENE_TOWN_SQUARE],
}

narrative_out = Output()
stats_html = HTML()
action_buttons = HBox(layout=Layout(flex_flow='wrap', width='100%'))


def top_traits(vector, n=5):
    arr = np.array(vector, float)
    idx = np.argsort(-arr)[:n]
    return [(traits[i], float(arr[i])) for i in idx]


def render_stats():
    vec = state['vector']
    level = 1 + int(state['total_xp'] // 30)
    quest_region = (snapshot.get('quests') or {}).get('main_ch1')
    quest_ok = in_quest_region(vec, quest_region, traits) if quest_region else False
    top = top_traits(vec, n=min(5, len(traits)))
    top_text = ', '.join([f"{name}={value:.2f}" for name, value in top]) if top else 'none'

    stats_html.value = (
        f"<b>Scene:</b> {state['scene_id']} | <b>NPC:</b> {SCENE_TO_NPC.get(state['scene_id'], '---')} "
        f"| <b>Level:</b> {level} | <b>Effort:</b> {state['effort_pool']:.2f} "
        f"| <b>Quest main_ch1:</b> {'available' if quest_ok else 'locked or undefined'}"
        f"<br/><b>Top traits:</b> {top_text}"
    )


def do_action(btn):
    global state
    kind = btn.action_kind

    if kind == 'train':
        event = {'type': EVENT_TRAINING, 'scene_id': state['scene_id']}
    elif kind == 'rest':
        event = {'type': EVENT_REST, 'scene_id': state['scene_id']}
    elif kind == 'travel':
        event = {'type': EVENT_TRAVEL, 'scene_id': btn.scene_id}
    elif kind == 'dialog':
        event = {'type': EVENT_DIALOG, 'scene_id': state['scene_id'], 'chosen_dialog': btn.dialog_label}
    else:
        return

    state, log_row = apply_one_event(state, event, snapshot=snapshot)
    event_history.append(log_row)

    with narrative_out:
        print(f"[{len(event_history)-1}] {event['type']} @ {event['scene_id']}")
        if kind == 'dialog':
            print(f"Dialog selected: {btn.dialog_label}")
        if log_row['warnings']:
            print('WARN:', '; '.join(log_row['warnings'][-3:]))
        if not log_row['force_applied']:
            print('No force data applied for this event (snapshot may be partial).')
        print('---')

    render_stats()
    refresh_buttons()


def refresh_buttons():
    children = []

    b_train = Button(description='Train')
    b_train.action_kind = 'train'
    b_train.on_click(do_action)
    children.append(b_train)

    b_rest = Button(description='Rest')
    b_rest.action_kind = 'rest'
    b_rest.on_click(do_action)
    children.append(b_rest)

    for scene_id in REACHABLE.get(state['scene_id'], []):
        b = Button(description=f'Travel to {scene_id}')
        b.action_kind = 'travel'
        b.scene_id = scene_id
        b.on_click(do_action)
        children.append(b)

    options = available_dialogs(state['vector'], state['scene_id'], snapshot=snapshot, trait_names=traits)
    for label, _, _ in options:
        b = Button(description=f'Dialog: {label}')
        b.action_kind = 'dialog'
        b.dialog_label = label
        b.on_click(do_action)
        children.append(b)

    if not options:
        b_disabled = Button(description='Dialog data unavailable', disabled=True)
        children.append(b_disabled)

    action_buttons.children = children


render_stats()
refresh_buttons()

with narrative_out:
    print('Alignment OK:', alignment['ok'])
    for warning in alignment['warnings']:
        print('WARN:', warning)
    print(SCENE_DESCRIPTIONS[state['scene_id']])
    print('Choose an action below.')

VBox([stats_html, narrative_out, action_buttons])
